In [1]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
import numpy as np
import json, time

In [2]:
filepath = "data/reviews.jl"
save_dir = "target/"
ALS_setting = {
    'rank': 10,
    'numIterations': 20
}

In [3]:
# Load and parse the data
data = sc.textFile(filepath)
ratings = data.map(lambda l: json.loads(l))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))).cache()

In [4]:
def splitRatings(ratings, bound=1000):
    c = ratings.count()
    print("Number of Reviews: %d"%c)
    print("_________________________________")
    train_ratings = ratings.filter(lambda r: r.user >= bound)
    test_ratings = ratings.filter(lambda r: r.user < bound)
    tmp = train_ratings.count()
    print("Training Data: %6.2f%% %d"%(float(tmp)/c*100, tmp))
    tmp = test_ratings.count()
    print("    Test Data: %6.2f%% %d"%(float(tmp)/c*100, tmp))
    return train_ratings, test_ratings

In [5]:
train_ratings, test_ratings = splitRatings(ratings)

Number of Reviews: 2685066
_________________________________
Training Data:  99.86% 2681301
    Test Data:   0.14% 3765


In [6]:
# Build the recommendation model using Alternating Least Squares
def cf_als(ratings, ALS_setting):
    print("Training CF Model")
    print("_________________________________")
    start_time = time.time()
    model = ALS.train(ratings, ALS_setting['rank'], ALS_setting['numIterations'])
    end_time = time.time()
    print "Time Cost: %.2fs"%(end_time - start_time)
    
    # Evaluate the model on training data
    print("Evaluating CF Model")
    print("_________________________________")
    start_time = time.time()
    testdata = ratings.map(lambda p: (p[0], p[1]))
    predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
    ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
    end_time = time.time()
    print("Mean Squared Error = " + str(MSE))
    print "Time Cost: %.2fs"%(end_time - start_time)
    
    return model

In [7]:
def saveModel(model, name):
    start_time = time.time()
    vBusiness = model.productFeatures()
    vBusiness.saveAsPickleFile(save_dir+name+"/vBusiness")
    vUser = model.userFeatures()
    vUser.saveAsPickleFile(save_dir+name+"/vUser")
    
    # Save and load model
    model.save(sc, save_dir+name+"/cfModel")

    end_time = time.time()
    print("Number of Business: %d"%vBusiness.count())
    print("Number of User: %d"%vUser.count())
    print "Time Cost: %.2fs"%(end_time - start_time)
    
def loadModel(name):
    start_time = time.time()
    # Load vUser
    vUser = sc.pickleFile(save_dir+name+"/vUser")
    # Load vBusiness
    vBusiness = sc.pickleFile(save_dir+name+"/vBusiness")
    # Load Model
    model = MatrixFactorizationModel.load(sc, save_dir+name+"/cfModel")
    end_time = time.time()
    print "Time Cost: %.2fs"%(end_time - start_time)
    return model, vUser, vBusiness

### Builde CF Model based on training data

In [8]:
trainModel = cf_als(train_ratings, ALS_setting)

Training CF Model
_________________________________
Time Cost: 66.05s
Evaluating CF Model
_________________________________
Mean Squared Error = 0.203936514774
Time Cost: 112.12s


In [10]:
saveModel(trainModel, "train")

KeyboardInterrupt: 

### Build the complete CF Model

In [13]:
model = cf_als(ratings, ALS_setting)

Training CF Model
_________________________________
Time Cost: 56.76s
Evaluating CF Model
_________________________________
Mean Squared Error = 0.201828733154
Time Cost: 119.29s


In [10]:
def getUserFeature(uid, ratings, vBusiness):
    def getMatrix(m):
        vb = []
        mA = []
        for r in m.collect():
            vb.append(r.label)
            mA.append(r.features.toArray())
        return np.array(vb), np.array(mA)
    businessDict = ratings.filter(lambda r: r.user == uid)\
                          .map(lambda r: (r.product, r.rating)).collectAsMap()
    compactM = vBusiness.filter(lambda r: r[0] in businessDict)\
                  .map(lambda r: LabeledPoint(businessDict[r[0]], r[1]))
    vb, mA = getMatrix(compactM)
    return np.linalg.inv(mA.T.dot(mA)).dot(mA.T).dot(vb)

def getKNN(vUser, vu, k):
    return vUser.map(lambda r: (r[0], np.linalg.norm(vu-r[1]))).top(k, key=lambda r: -r[1])

def getSimilarity(dv, v):
    return 1 - dv/np.linalg.norm(v)

def getEatingMate(uid, ratings, model, knn=5):
    vu = getUserFeature(uid, ratings, model.productFeatures().cache())
    knnUsers = getKNN(model.userFeatures().cache(), vu, knn)
    return map(lambda r: (
            r[0],
            getSimilarity(r[1], vu),
            map(lambda r: r.product, trainModel.recommendProducts(r[0], 5))
        ), knnUsers), vu

In [11]:
def evalEatingMateRecomd(uid, ratings, model):
    start_time = time.time()
    knnUsers, vu = getEatingMate(uid, ratings, model)
    for mid, sim, recomds in knnUsers:
        print("%6d %d%% %s"%(mid, int(sim*100), recomds))
    end_time = time.time()
    print "Time Cost: %.2fs"%(end_time - start_time)

In [46]:
knnUsers, vu = getEatingMate(2, test_ratings, trainModel)

In [12]:
vUser = trainModel.userFeatures().cache()
vBusiness = trainModel.productFeatures().cache()

In [13]:
a = vUser.filter(lambda r: r[0] == 396813).first()

In [15]:
a

(396813,
 array('d', [0.3815128803253174, 1.265856385231018, 0.5928857326507568, -0.20625562965869904, -0.4474773705005646, 1.4191725254058838, 0.42927369475364685, 0.144062340259552, -1.428943157196045, 0.9677742719650269]))

In [16]:
trainModel.recommendProducts(396813, 10)

[Rating(user=396813, product=79400, rating=13.423081043369281),
 Rating(user=396813, product=57958, rating=13.330221388776549),
 Rating(user=396813, product=54845, rating=13.04336914111168),
 Rating(user=396813, product=64471, rating=12.991971093562496),
 Rating(user=396813, product=6512, rating=12.950521639175474),
 Rating(user=396813, product=45989, rating=12.791118714882575),
 Rating(user=396813, product=23344, rating=12.756187686681308),
 Rating(user=396813, product=70608, rating=12.74609967160289),
 Rating(user=396813, product=74067, rating=12.433462019147804),
 Rating(user=396813, product=23360, rating=12.33840877075432)]

In [14]:
vBusiness.map(lambda r: (r[0], a.dot(r[1])))

[(4,
  array('d', [0.4100809097290039, 1.6258713006973267, -1.3126716613769531, -2.2342915534973145, 0.5001659989356995, 0.23033598065376282, 0.6178968548774719, 0.22995373606681824, 1.664929986000061, -1.4500586986541748]))]

In [56]:
vBusiness.map(lambda r: (r[0],r[1].dot(a))).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3696.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3696.0 (TID 671, localhost): java.lang.StackOverflowError
	at java.io.ObjectInputStream$BlockDataInputStream.read(ObjectInputStream.java:2728)
	at java.io.ObjectInputStream$BlockDataInputStream.readFully(ObjectInputStream.java:2752)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:1989)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:1924)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1801)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1351)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2000)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:1924)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.ja

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 59753)
----------------------------------------


Traceback (most recent call last):
  File "/Users/Zehao/anaconda/envs/py2/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/Zehao/anaconda/envs/py2/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/Users/Zehao/anaconda/envs/py2/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/Zehao/anaconda/envs/py2/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/usr/local/Cellar/apache-spark/2.0.2/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/Cellar/apache-spark/2.0.2/libexec/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


In [47]:
knnUsers

[(396813, 0.70288703989976098, [78179, 51136, 85299, 54658, 85329]),
 (192423, 0.67005040506902702, [54658, 51136, 85341, 1282, 1166]),
 (317732, 0.66362696723673631, [78179, 85329, 54658, 77005, 84388]),
 (461254, 0.65213646599495689, [51136, 85299, 54658, 85341, 1166]),
 (110819, 0.6403807254593914, [23523, 54658, 64527, 83625, 51136])]

In [41]:
evalEatingMateRecomd(2, test_ratings, trainModel)

ValueError: need more than 2 values to unpack

In [ ]:
evalEatingMateRecomd(2, test_ratings, model)